In [1]:
import asyncio
import websockets
import json
import pandas as pd
from csv import writer
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import time

In [2]:
msg = \
{
  "jsonrpc" : "2.0",
  "id" : 8772,
  "method" : "public/get_order_book",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL",
    "depth" : 10,
  }
}


In [5]:
keys = ['timestamp', 'best_bid_price', 'best_ask_price', 'estimated_delivery_price', 'index_price', 'mark_price', 'open_interest', 'settlement_price', 'change_id']
columns = keys + [x for i in range(10) for x in [f"ap_{i}", f"av_{i}", f"bp_{i}", f"bv_{i}"]]

old_lob_state = None

async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
    
    utc_timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    filename = f'_input/data/{utc_timestamp}.csv'

    with open(filename, 'w') as f:

        w = writer(f)
        w.writerow(columns)

        for i in tqdm(range(100000)):

            while True:
                
                await websocket.send(json.dumps(msg))
                data = await websocket.recv()
                data = json.loads(data)
                
                if 'result' not in data:
                    continue
                
                new_lob_state = [y for x in data['result']['bids'] + data['result']['asks'] for y in x]
                if old_lob_state != new_lob_state: break

            old_lob_state = new_lob_state

            vals = [data['result'][k] for k in keys]

            best_bid_price = data['result']['best_bid_price']
            best_ask_price = data['result']['best_ask_price']

            aps = [a[0] for a in data['result']['asks']]
            avs = [a[1] for a in data['result']['asks']]
            bps = [b[0] for b in data['result']['bids']]
            bvs = [b[1] for b in data['result']['bids']]

            vals += [x for ap, av, bp, bv in zip(aps, avs, bps, bvs) for x in [ap, av, bp, bv]]
            w.writerow(vals)

        f.close()

  0%|          | 239/100000 [00:34<4:03:22,  6.83it/s] 


ConnectionClosedOK: code = 1000 (OK), reason = over_limit